In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anime-recommendations-database/rating.csv
/kaggle/input/anime-recommendations-database/anime.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

In [4]:
anime_df = pd.read_csv("/kaggle/input/anime-recommendations-database/anime.csv")
rating_df = pd.read_csv("/kaggle/input/anime-recommendations-database/rating.csv")

# `Preprocessing the data`

In [5]:
rating_df = rating_df[rating_df.user_id <= 10000]

In [6]:
rating_df['rating'] = rating_df['rating'].replace(-1, pd.NA)
rating_df = rating_df.drop_duplicates(['user_id', 'anime_id'])

In [7]:
# Create a user-item interaction matrix
user_item_matrix = rating_df.pivot(index='user_id', columns='anime_id', values='rating')

In [8]:
user_item_matrix

anime_id,1,5,6,7,8,15,16,17,18,19,...,34048,34085,34103,34107,34136,34173,34240,34283,34324,34325
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,8,NaN,NaN,6,NaN,6,6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,10,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# `Collaborated Filtering`

In [9]:
user_similarity = cosine_similarity(user_item_matrix.fillna(0))

/tmp/ipykernel_33/2554419378.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  user_similarity = cosine_similarity(user_item_matrix.fillna(0))


In [10]:
def user_based_recommendations(user_id, user_item_matrix, user_similarity, n=5):
    # Get the similarity scores for the target user
    user_scores = user_similarity[user_id - 1]
    
    # Sort users by similarity in descending order
    similar_users = sorted(enumerate(user_scores), key=lambda x: x[1], reverse=True)
    
    # Initialize a list to store recommended anime
    recommended_anime = []
    
    for user, score in similar_users[1:]:  # Exclude the target user
        # Get the anime the similar user has rated positively
        rated_anime = user_item_matrix.loc[user_item_matrix.index[user - 1]]
        positively_rated_anime = rated_anime[rated_anime >= 6].index
        
        # Exclude anime the target user has already rated
        target_user_rated_anime = user_item_matrix.loc[user_id].dropna().index
        recommended_anime.extend(set(positively_rated_anime) - set(target_user_rated_anime))
        
        # Limit the number of recommendations
        if len(recommended_anime) >= n:
            break
    
    return recommended_anime[:n]

In [11]:
user_id = 10  # Replace with the user ID you want to test

# Test user-based recommendations
user_recommendations = user_based_recommendations(user_id, user_item_matrix, user_similarity, n=5)

In [12]:
print(f"User-Based Recommendations for User ID {user_id}:")
for anime_id in user_recommendations:
    anime_name = anime_df.loc[anime_df['anime_id'] == anime_id, 'name'].values[0]
    print(f"- {anime_name}")

User-Based Recommendations for User ID 10:
- Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.
- Coppelion
- Shokugeki no Souma
- Rail Wars!
- Death Billiards


In [13]:
# Calculate propensity scores
propensity_scores = rating_df.groupby('anime_id').size() / len(rating_df['user_id'].unique())
propensity_scores = propensity_scores.clip(lower=0.01)  # Avoid division by zero

# Create a user-item interaction matrix with IPW
user_item_matrix = rating_df.pivot(index='user_id', columns='anime_id', values='rating')

# Apply IPW
for anime_id in user_item_matrix.columns:
    user_item_matrix[anime_id] = user_item_matrix[anime_id] / propensity_scores[anime_id]

In [14]:
user_similarity = cosine_similarity(user_item_matrix.fillna(0))

def ipw_user_based_recommendations(user_id, user_item_matrix, user_similarity, propensity_scores, n=5):
    # Get the similarity scores for the target user
    user_scores = user_similarity[user_id - 1]
    
    # Sort users by similarity in descending order
    similar_users = sorted(enumerate(user_scores), key=lambda x: x[1], reverse=True)
    
    # Initialize a dictionary to store recommended anime and their weighted scores
    recommended_anime = {}
    
    for user, score in similar_users[1:]:  # Exclude the target user
        # Get the anime the similar user has rated positively
        rated_anime = user_item_matrix.loc[user_item_matrix.index[user]]
        positively_rated_anime = rated_anime[rated_anime >= 6].index
        
        # Exclude anime the target user has already rated
        target_user_rated_anime = user_item_matrix.loc[user_id].dropna().index
        new_recommendations = set(positively_rated_anime) - set(target_user_rated_anime)
        
        # Add weighted scores to recommendations
        for anime_id in new_recommendations:
            if anime_id not in recommended_anime:
                recommended_anime[anime_id] = score / propensity_scores[anime_id]
            else:
                recommended_anime[anime_id] += score / propensity_scores[anime_id]
    
    # Sort recommendations by weighted score and return top n
    sorted_recommendations = sorted(recommended_anime.items(), key=lambda x: x[1], reverse=True)
    return [anime_id for anime_id, _ in sorted_recommendations[:n]]

# Test the IPW-based recommender
user_id = 10  # Replace with the user ID you want to test
ipw_recommendations = ipw_user_based_recommendations(user_id, user_item_matrix, user_similarity, propensity_scores, n=5)

print(f"IPW-Based Recommendations for User ID {user_id}:")
for anime_id in ipw_recommendations:
    anime_name = anime_df.loc[anime_df['anime_id'] == anime_id, 'name'].values[0]
    print(f"- {anime_name}")

/tmp/ipykernel_33/4009998704.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  user_similarity = cosine_similarity(user_item_matrix.fillna(0))


IPW-Based Recommendations for User ID 10:
- Akame ga Kill!
- Shingeki no Kyojin
- Hunter x Hunter (2011)
- Tokyo Ghoul √A
- Tokyo Ghoul
